In [ ]:
#Import statements
from int_dir.config_file import *
from helper_modules.ai_db_connect import *
from core_modules.create_target_scoring_tables import * 
import core_modules.create_intermediate_tables as dm_table_creation
from core_modules.create_features import *
from core_modules.create_intermediate_tables import *

import time
from Step_2_Model_Train import *

In [ ]:
# Run Model
start = time.time()
rf,all_predictions_df = run_model()
end = time.time()
print(end-start)

In [ ]:
# Find predictions at nth percentile
def pred_ints(rf, x_test, percentile):
        err_down = []
        err_up = []
        for x in range(len(x_test)):
            preds = []
            for pred in rf.estimators_:
                preds.append(pred.predict(np.reshape(x_test.iloc[x,:].array,(1,72)))[0])                
            err_down.append(np.percentile(preds, (100 - percentile) / 2. ))
            err_up.append(np.percentile(preds, 100 - (100 - percentile) / 2.))
        return err_down, err_up
    
err_down_95, err_up_95 = pred_ints(rf, x_test, percentile=95)

In [ ]:
# Combinate everything above to create df with predictions, actual etc
predictions = rf.predict(x_test)
df = pd.DataFrame({'Predicted_price':predictions,
                   'error_down_95':err_down_95,
                   'error_up_95':err_up_95,
                   })
f = pd.concat([test_key.reset_index(drop=True),
               x_test.reset_index(drop=True),
               y_test.reset_index(drop=True),
               df.reset_index(drop=True)], axis=1)

f['estimated_margin'] = f['Predicted_price'] - f['committed_cost_ty']
f['margin_lower'] = f['error_down_95'] - f['committed_cost_ty']
f['margin_upper'] = f['error_up_95'] - f['committed_cost_ty']

price_recomm = f[['key','Predicted_price',"error_down_95","error_up_95",
                  "estimated_margin","margin_lower","margin_upper"]]

# Write output to shiny dir for it to be used by shiny R scripts
price_recomm.to_csv("/home/ncod/Sanjog/code/b2b_bid_pricing/shiny_dashboard/input_data/price_recomm_20200124.csv",index=False)

In [ ]:
# Find predictions for every tree

# x_test = x_test.reset_index(drop = True).iloc[:25,:]
# y_test = y_test.reset_index(drop = True).iloc[:25,:]
# test_key = test_key.reset_index(drop = True).iloc[:25,:]

predictions = pd.DataFrame({'Predicted_price': rf.predict(x_test)})

def pred_ints(rf, x_test):
    vals = []
    for x in range(len(x_test)):
        preds = []
        for pred in rf.estimators_:
            preds.append(pred.predict(np.reshape(x_test.iloc[x,:].array,(1,72)))[0])  # 72 is # of features (change if needed)              
        vals.extend(preds)
    return vals
    
op = pred_ints(rf, x_test)
op = pd.DataFrame(np.reshape(np.array(op),(15509,500)))
prob = pd.concat([test_key.reset_index(drop=True),
                  y_test.reset_index(drop=True),
                  predictions.reset_index(drop=True),
                  op.reset_index(drop=True)], axis=1)

# Write output to shiny dir for it to be used by shiny R scripts
prob.to_csv("/home/ncod/Sanjog/code/b2b_bid_pricing/shiny_dashboard/input_data/price_alltreepredictions_0127.csv",index=False)

In [ ]:
# Temporary script to write any sql table to csv in platform repository

from int_dir.config_file import *

def export_keydetails(engine):

    query_data = "SELECT * FROM intermediate_data.key_2019details_2020_01_28"

    raw_data = pd.read_sql(query_data,engine)
    
    raw_data.to_csv("/home/ncod/dashboard_input_data/raw_files/key_2019details_referencetable_0128.csv",index=False)
    return

start = time.time()
engine = conn_eng()
export_keydetails(engine)
end = time.time()
print(end-start)